In [1]:

import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/CARS/tabular_data.csv', index_col=0)

In [2]:
# Python code to preprocess every column of a DataFrame by replacing NaNs with the mean of the column
import pandas as pd
from sklearn.impute import SimpleImputer
# Assuming 'df' is your DataFrame
# df = pd.read_csv('your_data.csv')  # Example to load a DataFrame

# Create an imputer object with a mean filling strategy
imputer = SimpleImputer(strategy='mean')

# Apply the imputer to our data frame
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [3]:
tabular_data = df_imputed

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix, brier_score_loss
from sklearn.model_selection import train_test_split


graph_data = pd.read_csv('/content/drive/MyDrive/CARS/tensor.csv')
X_graph = graph_data.iloc[:, :-1].values
y_graph = graph_data['trojan'].values


#tabular_data = pd.read_csv('actual_table_22_16features.csv')
X_tabular = tabular_data.iloc[:, :-1].values
y_tabular = tabular_data['trojan'].values



In [5]:
len(y_tabular)

9800

In [12]:
X_graph = np.expand_dims(X_graph, axis=2)

X_graph_train, X_graph_test, y_graph_train, y_graph_test = train_test_split(X_graph, y_graph, test_size=0.2, random_state=42)
X_tabular_train, X_tabular_test, y_tabular_train, y_tabular_test = train_test_split(X_tabular, y_tabular, test_size=0.2, random_state=42)


In [14]:
def create_convolutional_layers(input_tensor):
    conv1 = tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu')(input_tensor)
    pool1 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv1)
    return pool1

def create_dense_layers(input_tensor):
    dense1 = tf.keras.layers.Dense(64, activation='relu')(input_tensor)
    return dense1

graph_input = tf.keras.layers.Input(shape=(X_graph.shape[1], X_graph.shape[2]))
graph_conv_layers = create_convolutional_layers(graph_input)

graph_flat = tf.keras.layers.Flatten()(graph_conv_layers)

tabular_input = tf.keras.layers.Input(shape=(X_tabular.shape[1],))
tabular_dense_layers = create_dense_layers(tabular_input)

concatenated = tf.keras.layers.concatenate([graph_flat, tabular_dense_layers])

x = tf.keras.layers.Dense(128, activation='relu')(concatenated)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(3, activation='softmax')(x)  # 'num_classes' is the number of classes in your multi-class problem

model = tf.keras.models.Model(inputs=[graph_input, tabular_input], outputs=output)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [16]:
from keras.utils import to_categorical

# Assuming 'y_graph_train' contains class labels (e.g., 0, 1, 2, ...) for multi-class classification
y_graph_train_encoded = to_categorical(y_graph_train, num_classes=3)

model.fit([X_graph_train, X_tabular_train], y_graph_train_encoded, epochs=20, batch_size=5, verbose=1)


Epoch 1/20
1568/1568 [==============================] - 10s 5ms/step - loss: 846732535529472.0000 - accuracy: 0.8481
Epoch 2/20
1568/1568 [==============================] - 8s 5ms/step - loss: 187886338048.0000 - accuracy: 0.8819
Epoch 3/20
1568/1568 [==============================] - 4s 3ms/step - loss: 0.4551 - accuracy: 0.8825
Epoch 4/20
1568/1568 [==============================] - 6s 4ms/step - loss: 0.4429 - accuracy: 0.8827
Epoch 5/20
1568/1568 [==============================] - 4s 3ms/step - loss: 0.5145 - accuracy: 0.8821
Epoch 6/20
1568/1568 [==============================] - 4s 2ms/step - loss: 0.4428 - accuracy: 0.8827
Epoch 7/20
1568/1568 [==============================] - 4s 3ms/step - loss: 0.4692 - accuracy: 0.8825
Epoch 8/20
1568/1568 [==============================] - 6s 4ms/step - loss: 0.4435 - accuracy: 0.8825
Epoch 9/20
1568/1568 [==============================] - 4s 2ms/step - loss: 0.4428 - accuracy: 0.8827
Epoch 10/20
1568/1568 [==============================] -

In [21]:
y_pred = model.predict([X_graph_test, X_tabular_test])
y_pred_classes = np.argmax(y_pred, axis=1)  # Get the predicted class indices

# Calculate accuracy and other metrics for multi-class classification
accuracy = accuracy_score(y_graph_test, y_pred_classes)
confusion = confusion_matrix(y_graph_test, y_pred_classes)
classification_rep = classification_report(y_graph_test, y_pred_classes)

print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Confusion Matrix:\n{confusion}')
print(f'Classification Report:\n{classification_rep}')


62/62 [==============================] - 0s 2ms/step
Accuracy: 88.06%
Confusion Matrix:
[[1726    0    0]
 [ 132    0    0]
 [ 102    0    0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.88      1.00      0.94      1726
           1       0.00      0.00      0.00       132
           2       0.00      0.00      0.00       102

    accuracy                           0.88      1960
   macro avg       0.29      0.33      0.31      1960
weighted avg       0.78      0.88      0.82      1960



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
from sklearn.metrics import brier_score_loss
from keras.utils import to_categorical

# Assuming 'y_graph_test' contains class labels (e.g., 0, 1, 2, ...) for multi-class classification
# Calculate the true class probabilities as one-hot encoded labels
y_true_one_hot = to_categorical(y_graph_test, num_classes=3)

# Predict class probabilities using your trained model
y_pred_probabilities = model.predict([X_graph_test, X_tabular_test])

# Calculate the Brier score for each class
brier_scores = []

for class_idx in range(3):  # Assuming you have 3 classes
    class_true_probs = y_true_one_hot[:, class_idx]
    class_pred_probs = y_pred_probabilities[:, class_idx]
    class_brier_score = brier_score_loss(class_true_probs, class_pred_probs)
    brier_scores.append(class_brier_score)

print(f'Brier Scores for Each Class:')
for class_idx, score in enumerate(brier_scores):
    print(f'Class {class_idx}: {score:.4f}')


62/62 [==============================] - 0s 4ms/step
Brier Scores for Each Class:
Class 0: 0.1052
Class 1: 0.0628
Class 2: 0.0494
